In [20]:
import xml.etree.ElementTree as ET
#list of all the malicious permissions in the xml file
mal_permission = []
#to store the index of different permission to create the model afterwards
counter=0
#malicious
#creating a dictionary to assign different indices to different permissions to create the model
index_permission = {}
for i in range(1,4050):
    #path for different xml files(manifest)
    file = "D:\Code\BTL\BTL_Python\Train_Data\malicious\AndroidManifest(" + str(i) + ").xml"
    tree = ET.parse(file)
    root = tree.getroot()
    temp = []
    for item in root.iter('uses-permission'):
        x = item.get("{http://schemas.android.com/apk/res/android}name")
        #permission is of the form android.permission.PERMISSION_NAME(so using split to get the permission name only)
        x=x.split('.')[-1]
        if x not in index_permission : 
            index_permission[x] = counter
            counter += 1
        temp.append(x)
    if len(temp) != 0 : mal_permission.append(temp)




In [21]:
#BENIGN PERMISSION LIST
ben_permission=[]
for i in range(1,4780):
    try:
        file = "D:\Code\BTL\BTL_Python\Train_Data\\benign\AndroidManifest(" + str(i) + ").xml"
        tree = ET.parse(file)
        root = tree.getroot()
        temp = []
        for item in root.iter('uses-permission'):
            x = item.get("{http://schemas.android.com/apk/res/android}name")
            x=x.split('.')[-1]
            if x not in index_permission : 
                index_permission[x] = counter
                counter += 1
            temp.append(x)
        if len(temp) != 0 : ben_permission.append(temp)
    except:
        continue

In [22]:
len(ben_permission), counter,len(mal_permission)

(4718, 923, 4012)

In [23]:
index_permission

{'INTERNET': 0,
 'READ_EXTERNAL_STORAGE': 1,
 'READ_PHONE_STATE': 2,
 'ACCESS_COARSE_LOCATION': 3,
 'ACCESS_FINE_LOCATION': 4,
 'ACCESS_NETWORK_STATE': 5,
 'ACCESS_LOCATION_EXTRA_COMMANDS': 6,
 'WRITE_EXTERNAL_STORAGE': 7,
 'ACCESS_WIFI_STATE': 8,
 'CHANGE_WIFI_STATE': 9,
 'INSTALL_PACKAGES': 10,
 'READ_SMS': 11,
 'WRITE_SMS': 12,
 'SET_WALLPAPER': 13,
 'GET_TASKS': 14,
 'RECEIVE_BOOT_COMPLETED': 15,
 'VIBRATE': 16,
 'CALL_PHONE': 17,
 'WAKE_LOCK': 18,
 'DISABLE_KEYGUARD': 19,
 'RECEIVE_SMS': 20,
 'SEND_SMS': 21,
 'RESTART_PACKAGES': 22,
 'WRITE_APN_SETTINGS': 23,
 'WRITE_CONTACTS': 24,
 'BROADCAST_STICKY': 25,
 'RECEIVE_MMS': 26,
 'READ_CONTACTS': 27,
 'READ_LOGS': 28,
 'WRITE_SETTINGS': 29,
 'BLUETOOTH': 30,
 'BLUETOOTH_ADMIN': 31,
 'READ_SYNC_SETTINGS': 32,
 'WRITE_SYNC_SETTINGS': 33,
 'GET_ACCOUNTS': 34,
 'FLASHLIGHT': 35,
 'HARDWARE_TEST': 36,
 'WRITE_SECURE_SETTINGS': 37,
 'READ_SECURE_SETTINGS': 38,
 'CAMERA': 39,
 'MODIFY_PHONE_STATE': 40,
 'ACCESS_WIMAX_STATE': 41,
 'CHANGE_WI

In [24]:
import numpy as np
#creating a structure to input the details of different apps(for input data to insert in different ml algorithms)
data = np.array([[0] * (counter+1)]*(len(ben_permission) + len(mal_permission)))
data.shape

(8730, 924)

In [25]:
#read the list of permission and create input data
j=0 #index for data
#malicious permissions
for i in range(len(mal_permission)):
    for k in mal_permission[i]:
        index=index_permission[k]
        data[j][index]=1
    data[j][-1]=1
    j+=1
#benign permissions
for i in range(len(ben_permission)):
    for k in ben_permission[i]:
        index=index_permission[k]
        data[j][index]=1
    data[j][-1]=0
    j+=1
data.shape

(8730, 924)

In [38]:
from sklearn import model_selection
from sklearn.metrics import accuracy_score
x=data[:,0:counter]
y=data[:,-1]
x,xt,y,yt=model_selection.train_test_split(x,y,random_state=0)
x.shape

(6547, 923)

In [59]:
from sklearn.ensemble import RandomForestClassifier
#BEST ACCURACY
clf_random100 = RandomForestClassifier(n_estimators = 150)
clf_random100.fit(x,y)
y_ran100_p = clf_random100.predict(xt)
accuracy_score(yt, y_ran100_p)  

0.923957856161246

In [54]:
#LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression
clf_logistic=LogisticRegression(solver='lbfgs', max_iter=1000)
clf_logistic.fit(x,y)
y_log_p=clf_logistic.predict(xt)
accuracy_score(yt,y_log_p)

0.9088410444342647

In [40]:
from sklearn.naive_bayes import GaussianNB
clf_naivebayes=GaussianNB()
clf_naivebayes.fit(x,y)
y_naivebayes_p=clf_naivebayes.predict(xt)
accuracy_score(yt,y_naivebayes_p)

0.6202473660100779

In [41]:
from sklearn import svm
clf_svm = svm.SVC()
clf_svm.fit(x,y)
y_svm_p = clf_svm.predict(xt)
accuracy_score(yt, y_svm_p)

0.9175446633073752

In [42]:
from sklearn.neighbors import KNeighborsClassifier
clf_KNN3 = KNeighborsClassifier(n_neighbors=3)
clf_KNN3.fit(x, y)
y_KNN3_p=clf_KNN3.predict(xt)
accuracy_score(yt,y_KNN3_p)

0.918918918918919

In [43]:
clf_KNN5 = KNeighborsClassifier(n_neighbors=5)
clf_KNN5.fit(x, y)
y_KNN5_p=clf_KNN5.predict(xt)
accuracy_score(yt,y_KNN5_p)

0.9180027485112231

In [44]:
clf_KNN10 = KNeighborsClassifier(n_neighbors=10)
clf_KNN10.fit(x, y)
y_KNN10_p=clf_KNN10.predict(xt)
accuracy_score(yt,y_KNN10_p)

0.9120476408612002

In [45]:
from sklearn.tree import DecisionTreeClassifier
clf_decisiontree = DecisionTreeClassifier()
clf_decisiontree.fit(x,y)
y_dec_p = clf_decisiontree.predict(xt)
accuracy_score(yt, y_dec_p)

0.9102153000458085

In [60]:
#Export Model
import joblib
joblib.dump(clf_random100, "D:\Code\BTL\BTL_Python\Models\Random_Forest_Model_2.joblib")

['D:\\Code\\BTL\\BTL_Python\\Models\\Random_Forest_Model_2.joblib']